In [1]:
import os, os.path
import pathlib
import random
import sys

import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib
os.getcwd()
INSPECTION_PATH = pathlib.Path(os.getcwd()).parent / "country-backend"
sys.path.append(str(INSPECTION_PATH.absolute()))
from country.model.predict import model
from country.model.predict import preprocess, prediction, predict_image
from country.explainer.new_lime_ import explain_image, explain_image_for_seg_occ_test, create_segments
from PIL import Image

2022-02-26 18:21:28.677866: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
#load images
images = []

pathimages = ["/Users/theresatroger/z_GTC_Results/Umfrage_welches_Modell/Images/schwer/neu"]
pathresults = "/Users/theresatroger/z_GTC_Results/Umfrage_welches_Modell/explanations/20220224_explanations/" 

for path in pathimages:
	for f in os.listdir(path):
		if f.endswith(".png"):
			images.append(cv2.imread(os.path.join(path,f)))


In [2]:
# test 01_ seg_occ:
# count occ of each segment of one image for N runs

#testimage
image = cv2.imread("/Users/theresatroger/z_GTC_Results/Images1/Berlin/Berlin1.png")

# variable parameter options
seg_method = "slic" 		# Options: [slic, felzenszwalb, quickshift, watershed]
seg_settings = {}			# Options: slic -> n_segments: [100, 150, 250, 350, 400] #ursprünglich 250
							#			felzenszwalb -> 
num_samples = 700			# Options: 
samples_p = 0.5

# setting
runs_for_one_img = 4
model_ = model
threshold = 0.5
volume = 20

#methods
def seg_occ(img:np.ndarray)->np.ndarray:

		image_indices = explain_image_for_seg_occ_test(img = img, seg_method = "slic", seg_settings = {}, num_of_samples = 700, samples_p = 0.5,
						model_ = model_, threshold = threshold, volume = volume, colour =  "green", transparency = 0.7, segment_mask = segment_mask)
		return image_indices


#for N images
#...

#call segmentation method und use it for one image -> same segmentation foor each run
image = np.squeeze(preprocess(image))
segment_mask = create_segments(img=image, seg_method=seg_method, settings=seg_settings)
number_segments = np.unique(segment_mask).__len__()
print (number_segments)
# results indices
indices_occ = np.zeros(number_segments)
#for: amount of runs for one single image
for r in range(runs_for_one_img):
	indices = seg_occ(image)
	for i in indices:
		indices_occ[i] += 1
metric_absolut = 0
for s in indices_occ:
	metric_absolut += abs(s - runs_for_one_img/2)
metric_relative = metric_absolut/((runs_for_one_img/2)*indices_occ.__len__())
print (metric_relative)


106
0.7641509433962265


In [ ]:
x = np.arange(number_segments)

print (x)
print (indices_occ)
plt.scatter(x, indices_occ)
plt.show()